#### Car Evaluation Model

Descrição: O dataset contém diversas informações sobre carros, as classes unacc (unaacceptable), acc (acceptable), good, vgood (very good) e um total de 1728 intâncias

Resultados: O modelo se saiu bem em todas as métricas avaliadas e saiu do treinamento antes do final das épocas pelo early stopping.

In [1]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


In [2]:
# --- 1. Carregar e Inspecionar o Dataset ---
print("--- 1. Carregando e Inspecionando o Dataset Car Evaluation ---")
car_evaluation = fetch_ucirepo(id=19)
X_original = car_evaluation.data.features
y_original = car_evaluation.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

--- 1. Carregando e Inspecionando o Dataset Car Evaluation ---
Dimensão inicial de X: (1728, 6)
Colunas de X e seus tipos de dados:
buying      object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
dtype: object

Primeiras 5 linhas de X:
  buying  maint doors persons lug_boot safety
0  vhigh  vhigh     2       2    small    low
1  vhigh  vhigh     2       2    small    med
2  vhigh  vhigh     2       2    small   high
3  vhigh  vhigh     2       2      med    low
4  vhigh  vhigh     2       2      med    med

Classes de y:
class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64
--------------------------------------------------


In [3]:
# --- 2. Pré-processamento: One-Hot Encoding para Atributos Categóricos ---
print("--- 2. Pré-processamento: One-Hot Encoding ---")

# O Car Evaluation tem todas as colunas como 'object' (categóricas).
# O OneHotEncoder lidará com todas elas.
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso
# sparse_output=False é o mesmo que chamar .toarray() depois.
# Não precisamos do .astype(int).tolist() ainda, faremos a conversão de tipo para np.uint8 depois do split.
X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")
print("-" * 50)

--- 2. Pré-processamento: One-Hot Encoding ---
Dimensão de X após One-Hot Encoding: (1728, 21)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]]
--------------------------------------------------


In [4]:
# --- 3. Divisão dos Dados em Treino e Teste ---
print("--- 3. Dividindo os Dados em Treino e Teste ---")

# Para o WiSARD, os dados de entrada X devem ser do tipo np.uint8.
# E os rótulos y devem ser strings.
# Fazemos a conversão de tipo após o split para manter a consistência do tipo de dados.
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")
print("-" * 50)

--- 3. Dividindo os Dados em Treino e Teste ---
Tamanho do conjunto de treino (X_train): 518 amostras, 21 bits
Tamanho do conjunto de teste (X_test): 1210 amostras, 21 bits
Tipo dos elementos de X_train: uint8
--------------------------------------------------


In [26]:
# --- 4. Configuração e Treinamento do Modelo WiSARD ---
print("--- 4. Configurando e Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 8 # Ajuste este valor. 2 ou 3 são bons pontos de partida.

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
print("Treinando o WiSARD...")
wsd.train(X_train, y_train)
print("Treinamento concluído.")
print("-" * 50)


# --- 5. Classificação e Avaliação do Modelo WiSARD ---
print("--- 5. Classificação e Avaliação do Modelo WiSARD ---")

print("Classificando os dados de teste...")
predictions_str = wsd.classify(X_test)
print("Classificação concluída.")

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) # Converte o y_test também para numpy array para comparação elemento a elemento

print("Relatório de Classificação:")
print(classification_report(
    y_test_np, predictions_np,
    labels=np.unique(y_test)
))

--- 4. Configurando e Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=8
Treinando o WiSARD...
Treinamento concluído.
--------------------------------------------------
--- 5. Classificação e Avaliação do Modelo WiSARD ---
Classificando os dados de teste...
training 261Classificação concluída.
Relatório de Classificação:
training 518 of 518
classifying 58              precision    recall  f1-score   support

         acc       0.70      0.66      0.68       269
        good       0.67      0.12      0.21        48
       unacc       0.88      0.96      0.92       847
       vgood       0.79      0.33      0.46        46

    accuracy                           0.84      1210
   macro avg       0.76      0.52      0.57      1210
weighted avg       0.83      0.84      0.82      1210

classifying 1210 of 1210ying 624 of 1210
